# Data Cleaning:

Load Data from CSV (saved in part 2a) into Pandas Data Frame and read the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('friends_transcripts_s1_10.csv')

In [3]:
df.head()

,Unnamed: 0,Season,Episode,Title,Character,Line
0,0,season1,101,The One Where Monica Gets a New Roomate (The P...,Written by:,Marta Kauffman & David Crane
1,1,season1,101,The One Where Monica Gets a New Roomate (The P...,Monica:,There's nothing to tell! He's just some guy I...
2,2,season1,101,The One Where Monica Gets a New Roomate (The P...,Joey:,"C'mon, you're going out with the guy! There's..."
3,3,season1,101,The One Where Monica Gets a New Roomate (The P...,Chandler:,"All right Joey, be nice. So does he have a h..."
4,4,season1,101,The One Where Monica Gets a New Roomate (The P...,Phoebe:,"Wait, does he eat chalk?"


Review the data to determine how to clean

In [4]:
df.columns

Index(['Unnamed: 0', 'Season', 'Episode', 'Title', 'Character', 'Line'], dtype='object')

In [5]:
#Drop the first column as it is the old index
df.drop(axis=1, columns='Unnamed: 0', inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61116 entries, 0 to 61115
Data columns (total 5 columns):
Season       61116 non-null object
Episode      61116 non-null int64
Title        60013 non-null object
Character    61116 non-null object
Line         60017 non-null object
dtypes: int64(1), object(4)
memory usage: 2.3+ MB


In [8]:
#See if there are any null lines
df.isnull().sum()

Season          0
Episode         0
Title        1103
Character       0
Line         1099
dtype: int64

In [26]:
nulls = df[df.Line.isnull()]

The above tells me that there were some errors in scraping the data

1. Lines from Season 3 episode 2 was not picked up at all (about 280 lines)
1. Lines from Season 4 episode 6 was not picked up at all (about 247 lines)
1. 416 (2 lines)
1. 417 (9 lines)
1. 419 (4 lines)
1. 913 (all) and more

Therefore, I will count and determine if the distribution of the null lines for each of the main characters makes a material difference on their total lines

In [10]:
#Make a list of the main characters
main_characters = ['Ross', 'Joey', 'Chandler', 'Monica', 'Rachel', 'Phoebe']

main = {'Ross': ['Ross', 'ROSS', 'Ross ', 'Young Ross'],
        'Chandler': ['Chandler', 'Chan', 'CHAN', 'CHANDLER'],
        'Joey': ['Joey', 'JOEY', 'Fat Joey'],
        'Monica': ['Monica','MNCA', 'MONICA', 'MOnica', 'Young Monica'],
        'Phoebe': ['French Phoebe', 'PHOE', 'PHOEBE', 'Phoebe', 'Phoebe '],
        'Rachel': ['RACH', 'RACHEL', 'Racel', 'Rache', 'Rachel', 'Rachel ']}

In [11]:
#Replace the : for each character in the character column of the data frame
df.Character.replace(':', '', inplace=True)

In [12]:
df['Character'] = [x.replace(':', '') for x in df.Character]

In [13]:
#Print out a list of all the unique characters, to get an understanding of how many there are
set(df.Character)

{'A Casino Boss',
 'A Crew Member',
 'A Disembodied Voice',
 'A Drunken Gambler',
 'A Female Student',
 'A Male Customer',
 'A Student',
 'A Tourist',
 'A Waiter',
 'A Waiter in Drag',
 'A Woman',
 'A piece of something',
 'ALL',
 'AMBER',
 'AMGER',
 'Actor',
 'Adoption Agency guy',
 'Adrienne',
 'Agency guy',
 'Air stewardess',
 'Aired',
 'Airline Employee',
 'Alan',
 'Alex',
 'Alexandra Steele',
 'Alice',
 'All',
 'Allesandro',
 'Allison',
 'Amanda',
 'Amy',
 'Anchorwoman',
 'Andrea',
 'Angela',
 'Annabelle',
 'Announcer',
 'Another Scientist',
 'Another Tour Guide',
 'Another extra',
 'Answering Machine',
 'Anxious Wedding Guest',
 'Arthur',
 'Ashley',
 'Assistant',
 'Attendant',
 'Aunt Iris',
 'Aunt Lillian',
 'Aunt Lisa',
 'Aunt Millie',
 'Aurora',
 'BEN',
 'BIG BULLY',
 'BOTH',
 'Ballerina',
 'Bandleader',
 'Bank Officer',
 'Barry',
 'Bass Singer',
 'Ben',
 'Benjamin',
 'Bernice',
 'Big Nosed Rachel',
 'Bill',
 'Billy',
 'Billy Crystal',
 'Bitsy',
 'Bitter lady',
 'Bitter woman',

Since there are so many characters, determine whether our dataset would have enough data just using lines from the main characters.  

In [14]:
df.Character.isin(main_characters).sum()

47689

There are about 47k lines for the main characters out of the 61k scraped, which is about 77% of the data. I am comfortable with removing the other characters. Furthermore, I have not included any lines where more than one person (including a main character) was speaking as it seemed immaterial. See an example below.

In [15]:
df.Character[18415]

'Joey and Monica'

In [16]:
main.items()

dict_items([('Ross', ['Ross', 'ROSS', 'Ross ', 'Young Ross']), ('Chandler', ['Chandler', 'Chan', 'CHAN', 'CHANDLER']), ('Joey', ['Joey', 'JOEY', 'Fat Joey']), ('Monica', ['Monica', 'MNCA', 'MONICA', 'MOnica', 'Young Monica']), ('Phoebe', ['French Phoebe', 'PHOE', 'PHOEBE', 'Phoebe', 'Phoebe ']), ('Rachel', ['RACH', 'RACHEL', 'Racel', 'Rache', 'Rachel', 'Rachel '])])

In [17]:
def name_func(char):
    if char in main['Ross']:
        return 'Ross'
    elif char in main['Rachel']:
        return 'Rachel'
    elif char in main['Chandler']:
        return 'Chandler'
    elif char in main['Joey']:
        return 'Joey'
    elif char in main['Monica']:
        return 'Monica'
    elif char in main['Phoebe']:
        return 'Phoebe'
    else:
        return 'other'

In [18]:
name_func('MONICA')

'Monica'

In [19]:
updated_charnames = []
for char in df.Character:
    updated_charnames.append(name_func(char))

In [20]:
len(updated_charnames), len(df.Character)

(61116, 61116)

In [21]:
set(updated_charnames)

{'Chandler', 'Joey', 'Monica', 'Phoebe', 'Rachel', 'Ross', 'other'}

In [22]:
df['Character'] = updated_charnames

In [23]:
char_list = set(df.Character)

In [24]:
nulls[:5]

,Season,Episode,Title,Character,Line
3980,season1,118,The One With All The Poker,Ross:,NaN
8244,season3,302,The One Where No-one's Ready,Joey:,NaN
8245,season3,302,The One Where No-one's Ready,Chandler:,NaN
8246,season3,302,The One Where No-one's Ready,Joey:,NaN
8247,season3,302,The One Where No-one's Ready,Chandler:,NaN


In [27]:
#Count of null values for each of the main characters
nulls.groupby('Character').count()

,Season,Episode,Title,Line
Character,,,,
Chandler,144,144,144,0
Joey,153,153,153,0
Monica,99,99,99,0
Phoebe,103,103,103,0
Rachel,125,125,125,0
Ross,206,206,206,0
other,269,269,269,0


In [28]:
df[df.Season == 'season2'].groupby('Episode').count()[:10]

,Season,Title,Character,Line
Episode,,,,
201,218,218,218,218
202,204,204,204,204
203,204,204,204,204
204,215,215,215,215
205,283,283,283,283
206,231,231,231,231
207,210,210,210,210
208,213,213,213,213
209,243,243,243,243


Count of the total lines by character. We see that the season total is larger than the Line total, the line total takes into account the missing lines.

In [29]:
df.groupby('Character').count().loc[main_characters]

,Season,Episode,Title,Line
Character,,,,
Ross,9130,9130,8963,8924
Joey,8213,8213,8064,8060
Chandler,8397,8397,8277,8253
Monica,8378,8378,8224,8279
Rachel,9213,9213,9041,9088
Phoebe,7472,7472,7331,7369


In [30]:
df_test = df.groupby('Character')[['Season','Line']].count().loc[main_characters]

In [31]:
df_test['Pct_of_Total'] = df_test.Line / df_test.Season

In [32]:
df_test

,Season,Line,Pct_of_Total
Character,,,
Ross,9130,8924,0.977437
Joey,8213,8060,0.981371
Chandler,8397,8253,0.982851
Monica,8378,8279,0.988183
Rachel,9213,9088,0.986432
Phoebe,7472,7369,0.986215


From df_test, we see that there is no significant decrease in the total lines and overall, everyone stays around 97/98%

I will be dropping the null lines for the main characters in addition to dropping lines for non-main characters and lines that were said at the same time by more than one main character (i.e. Joey and Monica).

In [33]:
df_clean = df[~(df.Line.isnull())]

In [34]:
df_clean = df_clean[df_clean.Character.isin(main_characters)]

In [35]:
set(df_clean.Season)

{'10',
 'season1',
 'season2',
 'season3',
 'season4',
 'season5',
 'season6',
 'season7',
 'season8',
 'season9'}

In [36]:
df_clean['Season'] = [season.strip('season') for season in df_clean.Season]

In [37]:
df_clean.reset_index(inplace=True,drop=True)

In [38]:
df_clean.head()

,Season,Episode,Title,Character,Line
0,1,101,The One Where Monica Gets a New Roomate (The P...,Monica,There's nothing to tell! He's just some guy I...
1,1,101,The One Where Monica Gets a New Roomate (The P...,Joey,"C'mon, you're going out with the guy! There's..."
2,1,101,The One Where Monica Gets a New Roomate (The P...,Chandler,"All right Joey, be nice. So does he have a h..."
3,1,101,The One Where Monica Gets a New Roomate (The P...,Phoebe,"Wait, does he eat chalk?"
4,1,101,The One Where Monica Gets a New Roomate (The P...,Phoebe,"Just, 'cause, I don't want her to go through ..."


### Look at the Lines, is there anything to clean?
Will need to remove the words in between parentheses, in between <>, hrefs, and more, as they are not true dialogue 

In [39]:
!pip install regex
import regex

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Test Pattern

In [98]:
clean_lines = []
for text in df_clean.Line:
    pattern = regex.compile(r'\(\w*\s*[a-zA-Z\s]*\)*', regex.MULTILINE)
    clean_lines.append(regex.sub(pattern,'',text))

In [99]:
print(clean_lines[23])
print(df_clean.Line[23])

  Let me get you some coffee.
 (to Ross) Let me get you some coffee.


### Clean All Lines for Other Patterns

In [103]:
ln = df_clean.Line[43290]
print(ln)

Cool. </p></font><p align="center"> <strong> <font size="3">  END </font> </strong></p><hr/><p align="center"> <!--webbot bot="ImageMap" startspanrectangle=" (139,3) (197, 84) http://thecfsi.com/season2/216jmo.htm"rectangle=" (70,2) (131, 84) http://thecfsi.com/season2.htm"rectangle=" (0,0) (59, 80) http://thecfsi.com/season2/214towpv.htm" src="contrl08.gif"width="194" height="60" alt="contrl08.gif (14783 bytes)" --> <map name="FrontPageMap0"> <area coords="139, 3, 197, 84" href="http://thecfsi.com/season2/216jmo.htm" shape="RECT"/> <area coords="70, 2, 131, 84" href="http://thecfsi.com/season2.htm" shape="RECT"/> <area coords="0, 0, 59, 80" href="http://thecfsi.com/season2/214towpv.htm" shape="RECT"/> </map> <img alt="contrl08.gif (14783 bytes)" height="60" src="file:///D:/Friends%20Script%20Index/Season2/contrl08.gif" usemap="#FrontPageMap0" width="194"/> <!--webbot bot="ImageMap"i-checksum="3479" endspan --></p>


In [127]:
clean_lines = []
for text in df_clean.Line:
    pattern_1 = regex.compile(r'\(\w*\s*[a-zA-Z\s]*\)*', regex.MULTILINE)
    pattern_2 = regex.compile(r'[[<][\w*\s*\/*\d*\'*\,*\=*\"*\!*\-*\(*\:*\.*\%*\#*\+*\)*]+\>*\]*',regex.MULTILINE)
    pattern_3 = regex.compile(r'END', regex.MULTILINE)
    line1 = regex.sub(pattern_1,'',text)
    line2 = regex.sub(pattern_2,'',line1)
    line3 = regex.sub(pattern_3,'', line2)
    clean_lines.append(line3)

In [128]:
#Check one line
df_clean.Line[44528]

'Excellent.</font><p align="center"> <strong> <font size="3">  END </font> </strong></p><hr/><p align="center"> <!--webbot bot="ImageMap" startspanrectangle=" (139,1) (197, 84) http://thecfsi.com/season2/222towtp.htm"rectangle=" (70,3) (130, 83) http://thecfsi.com/season2.htm"rectangle=" (0,1) (60, 80) http://thecfsi.com/season2/220oyd.htm" src="contrl08.gif"width="194" height="60" alt="contrl08.gif (14783 bytes)" --> <map name="FrontPageMap0"> <area coords="139, 1, 197, 84" href="http://thecfsi.com/season2/222towtp.htm" shape="RECT"/> <area coords="70, 3, 130, 83" href="http://thecfsi.com/season2.htm" shape="RECT"/> <area coords="0, 1, 60, 80" href="http://thecfsi.com/season2/220oyd.htm" shape="RECT"/> </map> <img alt="contrl08.gif (14783 bytes)" height="60" src="file:///D:/Friends%20Script%20Index/Season2/contrl08.gif" usemap="#FrontPageMap0" width="194"/> <!--webbot bot="ImageMap"i-checksum="52661" endspan --></p>'

In [129]:
#Determine if the regex substitute cleaned the line appropriately.
clean_lines[44528]

'Excellent.              '

In [130]:
#Check the length of each list to ensure that all lines were captured
len(clean_lines), len(df_clean.Line)

(49973, 49973)

In [131]:
#Create a new column called "Clean Lines" and set it to the clean_lines list made above. 
df_clean['Clean_Lines'] = clean_lines

# Save Data Frame to CSV
To be used for part 3 - modeling

In [132]:
df_clean.to_csv('friends_transcripts_clean.csv', columns=['Season', 'Episode', 'Title', 'Character', 'Line', 'Clean_Lines'])